In [ ]:
# All code below is referenced from Lecture_1_2.ipynb provided by Gittu George for DSCI 525

In [80]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
from os import listdir


import pyarrow.feather as feather
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
from memory_profiler import memory_usage
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra

import altair as alt
from vega_datasets import data

In [8]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [9]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [10]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [11]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.41 s, sys: 3.62 s, total: 8.04 s
Wall time: 1min 9s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17 s, sys: 3.41 s, total: 20.4 s
Wall time: 24.5 s


In [9]:
#%%time
#df = pd.read_csv('figshareairline/ACCESS-CM2_daily_rainfall_NSW.csv')
#model_name = 'ACCESS-CM2'
#
#df['model'] = 'ACCESS-CM2'
#
#count = 0
#
#for f in listdir('figshareairline'):
#    if count != 0:
#        if '.csv' in f:
#            model_name = f.split('_daily')[0]
#            temp = pd.read_csv('figshareairline/' + f)
#            temp['model'] = model_name
#            df = pd.concat([df, temp], axis=0)
#    count += 1

CPU times: user 2min 35s, sys: 2min 22s, total: 4min 57s
Wall time: 5min 47s


## Combine data (using 5000 rows eachs)

In [12]:
%%time

#reduced size combined data
df = pd.read_csv('figshareairline/ACCESS-CM2_daily_rainfall_NSW.csv', nrows=5000)
model_name = 'ACCESS-CM2'

df['model'] = 'ACCESS-CM2'

count = 0

for f in listdir('figshareairline'):
    if count != 0:
        if '.csv' in f:
            model_name = f.split('_daily')[0]
            temp = pd.read_csv('figshareairline/' + f, nrows= 5000)
            temp['model'] = model_name
            df = pd.concat([df, temp], axis=0)
    count += 1

CPU times: user 503 ms, sys: 167 ms, total: 669 ms
Wall time: 1.04 s


In [146]:
#initiate dictionary to store timeit results
time_result = {}

### Save as csv

In [161]:
%%timeit -n1 -r1 -o
#save dataframe as csv
df.to_csv('data/combined_subset_data.csv')

1.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 1.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [32]:
time_result["Write_as_csv"] = _

### Load data as Arrow format

In [33]:
%%time
#load csv data using arrow 
dataset = ds.dataset('data/combined_subset_data.csv', format="csv")
# make arrow table format
table = dataset.to_table()

CPU times: user 59 ms, sys: 31.6 ms, total: 90.7 ms
Wall time: 133 ms


### Save as Parquet format

In [34]:
%%timeit -n1 -r1 -o
#save data table as parquet file
pq.write_to_dataset(table, 'data/combined_subset_data.parquet')

93.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 93.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [35]:
time_result["Write_as_parquet"] = _

### Save as feather format

In [36]:
%%timeit -n1 -r1 -o
#save data table as feather file
feather.write_feather(table, 'data/combined_subset_data.feather')

14 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 14 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [37]:
time_result["Write_as_feather"] = _

In [148]:
%%R

#load R packages
suppressWarnings(library(arrow))
suppressWarnings(library(dplyr))
suppressWarnings(library(tidyverse))

## Read in as Parquet

In [45]:
%%timeit -n1 -r1 -o
%%R
#read in parquet
#r_table_parq <- read_parquet('data/combined_data.parquet', col_select = ("lon_min"))

start_time <- Sys.time()
r_table <- arrow::read_parquet('data/combined_subset_data.parquet/753a862c21e74e2ba4154e859bda44cc.parquet')

76.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 76.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [46]:
time_result["Read_as_parquet"] = _

## Read in as feather 

In [48]:
%%timeit -n1 -r1 -o
%%R
#read in feather
start_time <- Sys.time()
r_table <- arrow::read_feather('data/combined_subset_data.feather')

end_time <- Sys.time()
print(end_time - start_time)

Time difference of 0.133743 secs
179 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 179 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [49]:
time_result["Read_as_feather"] = _

## Experimenting the exchange of data b/w python and R.
* Arrow exchange
* Pandas exchange


### Transfer pandas dataframe to R

In [52]:
%%timeit -o
df = pd.read_csv("data/combined_subset_data.csv")

171 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 171 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [53]:
result = _ 

In [55]:
%%timeit -n1 -r1 -o
%%R -i df
## here I am transferring the python dataframe to R
## Look how much time it takes ! or tell me how much time you end up waiting !!
# if you see here this operation takes less time inside the R terminal, but overall it takes time , 
## The rest of time in spent on serialization and deserialization

#start_time <- Sys.time()
#library(dplyr)
#print(class(df))
#result <- df %>% count(UniqueCarrier)
#print(result)
#end_time <- Sys.time()
#print(end_time - start_time)

NULL
4.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 4.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [56]:
time_result["pandas_exchange"] = _

### Arrow Exchange

In [162]:
dataset = ds.dataset("data/combined_subset_data.csv", format="csv")
table = dataset.to_table()

In [163]:
#r_table = pyra.converter.py2rpy(table)

In [164]:
speed = pd.DataFrame.from_dict(time_result, orient='index')
speed

""


In [155]:
%%sh
#size of the combined_data files in different formats
du -sh data/combined_subset_data.csv
du -sh data/combined_subset_data.feather
du -sh data/combined_subset_data.parquet

 13M	data/combined_subset_data.csv
2.4M	data/combined_subset_data.feather
1.5M	data/combined_subset_data.parquet


In [156]:
#create dataframe for plots
speed1 = pd.DataFrame({'method':["Write_as_csv", "Write_as_parquet", "Write_as_feather", "Read_as_parquet", "Read_as_feather", "pandas_exchange"], 
                       'time': [0.934, 0.0931, 0.014,  0.0768, 0.179, 4.54]})

size = pd.DataFrame({'format':["csv", "feather", "parquet"], 
                       'size': [13, 2.4, 1.5]})

In [159]:
#plots
plot_write = alt.Chart(speed1[0:3], title = "Write times with different file formats (Subsetted data)").mark_bar().encode(
    alt.X("time", title= "Time (seconds)"),
    alt.Y("method", sort='-x')
)

plot_read = alt.Chart(speed1[3:6],  title = "Read times with different file formats in R (Subsetted data)").mark_bar().encode(
    alt.X("time", title= "Time (seconds)"),
    alt.Y("method", sort='-x')
)

plot_size = alt.Chart(size,  title = "Size of file formats (Subsetted data)").mark_bar().encode(
    alt.X("size", title= "Size (MB)"),
    alt.Y("format", sort='-x')
)

plot_write & plot_read & plot_size

alt.VConcatChart(...)

**Parquet vs Feather**
* Parquet generally has a smaller size (takes up less disk space)
* feather should have faster read and write times (when using local disk)
    * parquet files need to be decoded to arrow format then convert to R or pandas
    * feather is already in arrow format
* When read over the network, we expect Parquet to outperform.
